# Clasificador de Tweets
### (Peliculas o Turismo)

A continuacion se ha hecho un identificador de tweets (Deportes o turismo)

La primera parte, consistio en crear una muestra o archivo que contiene tweets de deportes y otro de turismo. 
EL objetivo es dar cierta inteligencia para que pueda identificar cualquier otro tweet y clasificarlo en base al conocimiento adquirido. Los archivos de alimentacion son: **deportes.txt y turismo.txt**

En segunda parte, se bajaron 2000 tweets **(archivo JSONtweets.json)**, los cuales fueron ingresados y analizados, y el resultado es mostrado en pantalla al final del ejercicio.

In [1]:
import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
import datetime
import time
import csv
import json
import nltk
import re
from pprint import pprint

from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier
import re

Obteniendo los datos de Twitter (2000 registros)

In [2]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [3]:
def search(hashtag, Max_Tweets, Recdata):

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    data = Cursor(api.search, q=hashtag, lang='es').items(Max_Tweets)
    
    current_working_dir = "./data/"
    log_tweets = current_working_dir  + 'JSONtweets.json'
    with open(log_tweets, 'w') as outfile:
        for tweet in data:
            Recdata.append(json.loads(json.dumps(tweet._json)))
            outfile.write(json.dumps(tweet._json))
            outfile.write("\n")
            
    # Podemos Crear dataset con esta informacion
    tweets = pd.DataFrame()
    # Campos del Tweet
#    tweets['created_at'] = map(lambda tweet: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')), Recdata)
#    tweets['user'] = map(lambda tweet: tweet['user']['screen_name'], Recdata)
    tweets['text'] = map(lambda tweet: tweet['text'].encode('utf-8'), Recdata)
   
    # Generar un archivo cvs
    tweets.to_csv('./data/CVStweets.csv', sep=';', header=False)

In [4]:
#Max_Tweets = 1000
#tweets_data = []
#hashtag = "deportes"
#
#search(hashtag, Max_Tweets, tweets_data)

Cargamos los datos que guardamos en el archivo de Tweet, para no ir siempre a Twitter, aunque el codigo utilizado para la descarga de tweets, esta al inicio de esta practica.

Ahora cargamos los Tweets

In [5]:
tweet_list = []
with open('./data/JSONtweets.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tweet_list.append(tweet['text'].encode("utf-8"))

In [6]:
#tw2 = []
#puntuacion = '\n'
#
#for tweet in tweet_list:
#    no_punct = ""
#    for char in tweet:
#        if char not in puntuacion:
#            no_punct = no_punct + char
#    tw2.append(no_punct)

In [7]:
#csvfile = "./data/peliculas.txt"
#
##Assuming res is a flat list
#with open(csvfile, "w") as output:
#    writer = csv.writer(output, lineterminator='\n')
#    for val in tweet_list:
#        writer.writerow([val])    

In [8]:
#Proceso de Tweets
def processTweet(tweet):
    tweet = tweet.lower()
    # www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #@username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

stopWords = []

def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

In [9]:
def getFeatureVector(tweet):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector
#end

In [10]:
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

In [11]:
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def read_tweets(fname, t_type):
    tweets = []
    f = open(fname, 'r')
    line = f.readline()
    while line != '':
        tweets.append([line, t_type])
        line = f.readline()
    f.close()
    return tweets

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

def classify_tweet(tweet):
    return classifier.classify(extract_features(nltk.word_tokenize(tweet)))

In [12]:
# Lee los tweets de deportes y turismo
dep_tweets = read_tweets('./data/deportes.txt', 'DEPORTES')
tur_tweets = read_tweets('./data/turismo.txt', 'TURISMO')

In [13]:
tweets = []
for (words, sentiment) in dep_tweets + tur_tweets:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    tweets.append((words_filtered, sentiment))

word_features = get_word_features(get_words_in_tweets(tweets))

# get the training set and train the Naive Bayes Classifier
training_set = nltk.classify.util.apply_features(extract_features, tweets)
classifier = NaiveBayesClassifier.train(training_set)

In [14]:
# Calculando la eficiencia
test_tweets = read_tweets('./data/deportes_test.txt', 'DEPORTES')
test_tweets.extend(read_tweets('./data/turismo_test.txt', 'TURISMO'))
total = accuracy = float(len(test_tweets))

for tweet in test_tweets:
    if classify_tweet(tweet[0]) != tweet[1]:
        accuracy -= 1

ntottes = total + accuracy
print('Total accuracy: %f%% (%d/%d).' % (accuracy / total * 100, accuracy, len(test_tweets)))

Total accuracy: 92.857143% (13/14).


Ahora analiza todos los tweets bajados e indica si son peliculas o deportes

In [15]:
NBClassifier = nltk.NaiveBayesClassifier.train(training_set)
for tweet in tweet_list:
    processedTestTweet = processTweet(tweet)
    print tweet + '  ----->  ' + NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet))) + '\n'


RT @CaracolDeportes: ⏰80 G⚽⚽⚽L de Ayron Del Valle. Millonarios 3-1 Medellín, fecha 20 #LigaAguila https://t.co/nlG02bcqw9 https://t.co/Vvi9…  ----->  DEPORTES

RT @maldiniplus: Un ídolo. Un maestro. Conocerle fue una gran satisfacción. Siempre aprendiendo de él. 
https://t.co/MHmz89NXU3  ----->  DEPORTES

RT @leonelayamuca: Universitario de Deportes, del balompié peruano la máxima expresión  #YdaleU  ----->  DEPORTES

Mirá los goles de #Boca 1 - #RosarioCentral 1 #Benedetto #TeoGutierrez https://t.co/fMZMdUCFB6 vía @JornadaWeb https://t.co/yLtnDywmwb  ----->  DEPORTES

Schroder amplía la racha ganadora de los líderes Hawks  https://t.co/kBCuzNca3x #Deportes https://t.co/sPV3ZQgWAW  ----->  DEPORTES

Trump se reunió con un exrival, Mitt Romney https://t.co/GFR6E1zsxL #Deportes  ----->  DEPORTES

RT @ESPNDeportes: ¿Ya nos siguen en Snapchat? Miren quien esta por ahí... @iambeckyg @CarolinaPadron @osunaespn  👻espn_deportes  ----->  TURISMO

📺 #ANEnVivo Deportes Iquique se mantiene en el l